# How to use backpropagation to train a feedforward NN

This noteboo implements a simple single-layer architecture and forward propagation computations using julia libraries.

## Imports & Settings

In [ ]:
using CUDA, Flux, Random

Random.seed!(1234);

In [ ]:
results_path = "results";
!isdir(results_path) && mkdir(results_path);

## Input Data

### Generate random data

The target <code>y</code> represents two classes generated by two circular distribution that are not linearly separable because class 0 surrounds class 1.

We will generate 50,000 random samples in the form of two concentric circles with different radius using scikit-learn's make_circles function so that the classes are not linearly separable.

In [ ]:
N = 5000;
factor = 0.1;
noise = 0.1;

In [ ]:
n_iterations = 5000;
learning_rate = 0.0001;
momentum_factor = 0.5;

In [ ]:
# Adapted From SyntheticDatasets
using DataFrames
using PyCall
sk = pyimport("sklearn.datasets")

function convert(features::Array{T, 2}, labels::Array{D, 1})::DataFrame where {T <: Number, D <: Number}
    df = DataFrame()

    for i = 1:size(features)[2]
        df[!, Symbol("feature_$(i)")] = eltype(features)[]
    end
    
    df[!, :label] = eltype(labels)[]
    
    for label in unique(labels)
        for i in findall(r->r == label, labels)
            push!(df, (features[i, :]... , label))
        end
    end

    return df
end

function convert(features::Array{T, 2}, labels::Array{D, 2})::DataFrame where {T <: Number, D <: Number}
    df = DataFrame()

    for i = 1:size(features)[2]
        df[!, Symbol("feature_$(i)")] = eltype(features)[]
    end

    for i = 1:size(labels)[2]
        df[!, Symbol("label_$(i)")] = eltype(labels)[]
    end
    
    for row in 1:size(features)[1]
        push!(df, (features[row, :]... , labels[row, :]...))
    end
    
    return df
end


(features, labels) = sk.make_circles(
    n_samples=N,
    shuffle=true,
    noise=noise,
    factor=factor
);

circles = convert(features, labels);

In [ ]:

labels_column_name = "label"

y = circles[:, [labels_column_name]];
y = Matrix(y);

X = select!(circles, Not(labels_column_name));
X = Matrix(X);


In [ ]:
# define outcome matrix
Y = zeros(Float64, (N, 2));
for c ∈ [0, 1]
    # mask = collect(Iterators.flatten(y .== c))
    mask = vec(y .== c)
    Y[mask, c+1] .= 1.0
end

In [ ]:
"Shape of: X: $(size(X)) | Y: $(size(Y)) | y: $(size(y))"

### Visualize Data

In [ ]:
using PlotlyJS
using DataFrames

##### Set PlotlyJS Theme

In [ ]:
templates.default = "plotly_dark";
PlotlyJS.templates

In [ ]:
df = DataFrame(
    X1=X[:, 1],
    X2=X[:, 2],
    Label=y[:, 1];
);

ax = PlotlyJS.plot(
    df,
    x=:X1,
    y=:X2,
    m=1,
    color=:Label,
    kind="scatter",
    mode="markers",
    labels=Dict(
        :X1 => "X",
        :X2 => "Y",
        :Label => "Labels"
    ),
    marker=attr(size=8, line=attr(width=1, color="DarkSlateGrey")),
    PlotlyJS.Layout(
        title="Circles Dataset Visualization",
        width=600, height=600,
    )
)


## Neural Network Architecture

### Hidden Layer Activations

The hidden layer $h$ projects the 2D input into a 3D space. To this end, the hidden layer weights are a $2\times3$ matrix $\mathbf{W}^h$, and the hidden layer bias vector $\mathbf{b}^h$ is a 3-dimensional vector:

\begin{align*}
\underset{\scriptscriptstyle 2 \times 3}{\mathbf{W}^h} =
\begin{bmatrix} 
w^h_{11} & w^h_{12} & w^h_{13} \\
w^h_{21} & w^h_{22} & w^h_{23}
\end{bmatrix}
&& \underset{\scriptscriptstyle 1 \times 3}{\mathbf{b}^h} = 
\begin{bmatrix} 
b^h_1 & b^h_2 & b^h_3
\end{bmatrix}
\end{align*}

The output layer values $\mathbf{Z}^h$ result from the dot product of the $N\times\ 2$ input data $\mathbf{X}$ and the the $2\times3$ weight matrix $\mathbf{W}^h$ and the addition of the $1\times3$ hidden layer bias vector $\mathbf{b}^h$:

$$\underset{\scriptscriptstyle N \times 3}{\mathbf{Z}^h} = \underset{\scriptscriptstyle N \times 2}{\vphantom{\mathbf{W}^o}\mathbf{X}}\cdot\underset{\scriptscriptstyle 2 \times 3}{\mathbf{W}^h} + \underset{\scriptscriptstyle 1 \times 3}{\mathbf{b}^h}$$

The logistic sigmoid function $\sigma$ applies a non-linear transformation to $\mathbf{Z}^h$ to yield  the hidden layer activations as an $N\times3$ matrix:

$$\underset{\scriptscriptstyle N \times 3}{\mathbf{H}} = \sigma(\mathbf{X} \cdot \mathbf{W}^h + \mathbf{b}^h) = \frac{1}{1+e^{−(\mathbf{X} \cdot \mathbf{W}^h + \mathbf{b}^h)}} = \begin{bmatrix} 
h_{11} & h_{12} & h_{13} \\
\vdots & \vdots & \vdots \\
h_{N1} & h_{N2} & h_{N3}
\end{bmatrix}$$

In [ ]:
function logistic(z)
    1 / (1 + exp(-z))
end;

function hidden_layer(input_data, weights, bias)
    L = input_data * weights .+ bias
    result = logistic.(L)
    result
end;

### Output Activations

The values $\mathbf{Z}^o$ for the output layer $o$ are a $N\times2$ matrix that results from the dot product of the $\underset{\scriptscriptstyle N \times 3}{\mathbf{H}}$ hidden layer activation matrix with the $3\times2$ output weight matrix $\mathbf{W}^o$ and the addition of the $1\times2$ output bias vector $\mathbf{b}^o$:

$$\underset{\scriptscriptstyle N \times 2}{\mathbf{Z}^o} = \underset{\scriptscriptstyle N \times 3}{\vphantom{\mathbf{W}^o}\mathbf{H}}\cdot\underset{\scriptscriptstyle 3 \times 2}{\mathbf{W}^o} + \underset{\scriptscriptstyle 1 \times 2}{\mathbf{b}^o}$$

The Softmax function $\varsigma$ squashes the unnormalized probabilities predicted for each class  to lie within $[0, 1]$ and sum to 1.  The result is a $N\times2$ matrix with one column for each output class.

$$\underset{\scriptscriptstyle N \times 2}{\mathbf{\hat{Y}}} 
= \varsigma(\mathbf{H} \cdot \mathbf{W}^o + \mathbf{b}^o)
= \frac{e^{Z^o}}{\sum_{c=1}^C e^{\mathbf{z}^o_c}}
= \frac{e^{H \cdot W^o + \mathbf{b}^o}}{\sum_{c=1}^C e^{H \cdot \mathbf{w}^o_c + b^o_c}}
= \begin{bmatrix} 
\hat{y}_{11} & \hat{y}_{12}\\
\vdots & \vdots \\
\hat{y}_{n1} & \hat{y}_{n2} 
\end{bmatrix}$$

In [ ]:
function softmax(X)
    exp.(X) ./ sum(exp.(X), dims=2)
end;

In [ ]:
function output_layer(hidden_activations, weights, bias)
    softmax(hidden_activations * weights .+ bias)
end;


### Forward Propagation

The `forward_prop` function combines the previous operations to yield the output activations from the  input data as a function of weights and biases. The `predict` function produces the binary class predictions given weights, biases, and input data.

In [ ]:
function forward_prop(
    data,
    hidden_weights,
    hidden_bias,
    output_weights,
    output_bias)

    hidden_activations = hidden_layer(data, hidden_weights, hidden_bias)
    output_activations = output_layer(hidden_activations, output_weights, output_bias)
    output_activations
end;

function forward_prop(
    data,
    wab)

    hidden_weights, hidden_bias, output_weights, output_bias = wab

    hidden_activations = hidden_layer(data, hidden_weights, hidden_bias)
    output_activations = output_layer(hidden_activations, output_weights, output_bias)
    output_activations
end;

In [ ]:
function predict(
    data,
    hidden_weights,
    hidden_bias,
    output_weights,
    output_bias)

    y_predicted_proba = forward_prop(
        data,
        hidden_weights,
        hidden_bias,
        output_weights,
        output_bias
    )

    y_predicted_proba_rounded = round.(y_predicted_proba)

    y_predicted_proba_rounded
end;

function predict(
    data,
    trained_params)

    hidden_weights, hidden_bias, output_weights, output_bias = trained_params

    y_predicted_proba = forward_prop(
        data,
        hidden_weights,
        hidden_bias,
        output_weights,
        output_bias
    )

    y_predicted_proba_rounded = round.(y_predicted_proba)

    y_predicted_proba_rounded
end;

### Cross-Entropy Loss

The cost function $J$ uses the cross-entropy loss $\xi$ that sums the deviations of the predictions for each class $c$  $\hat{y}_{ic}, i=1,...,N$ from the actual outcome.

$$J(\mathbf{Y},\mathbf{\hat{Y}}) = \sum_{i=1}^n \xi(\mathbf{y}_i,\mathbf{\hat{y}}_i) = − \sum_{i=1}^N \sum_{i=c}^{C} y_{ic} \cdot log(\hat{y}_{ic})$$

In [ ]:
function loss(ŷ, y)
    # Binary Cross Entropy Loss
    # loss_values = -(y_true .* log.(y_hat)) .- ((1 .- y_true) .* log.(1 .- y_hat));
    loss_values = -(y .* log.(ŷ))
    sum(loss_values)
end;


## Backpropagation

Backpropagation updates parameters values based on the partial derivative of the loss with respect to that parameter, computed using the chain rule.

### Loss Function Gradient

The derivative of the loss function $J$ with respect to each output layer activation $\varsigma(\mathbf{Z}^o_i), i=1,...,N$, is a very simple expression:

$$\frac{\partial J}{\partial z^0_i} = \delta^o = \hat{y}_i-y_i$$

See [here](https://math.stackexchange.com/questions/945871/derivative-of-softmax-loss-function) and [here](https://deepnotes.io/softmax-crossentropy) for details on derivation.



In [ ]:
function loss_gradient(ŷᵢ, yᵢ)
    ŷᵢ - yᵢ
end

### Output Layer Gradients

#### Output Weight Gradients

To propagate the updates back to the output layer weights, we take the partial derivative of the loss function with respect to the weight matrix:

$$
\frac{\partial J}{\partial \mathbf{W}^o} = H^T \cdot (\mathbf{\hat{Y}}-\mathbf{Y}) = H^T \cdot \delta^{o}
$$

In [ ]:
function output_weight_gradient(H, loss_grad)
    H' * loss_grad
end;

#### Output Bias Update

To update the output layer bias values, we similarly apply the chain rule to obtain the partial derivative of the loss function with respect to the bias vector:

$$\frac{\partial J}{\partial \mathbf{b}_{o}} 
= \frac{\partial \xi}{\partial \mathbf{\hat{Y}}} \frac{\partial \mathbf{\hat{Y}}}{\partial \mathbf{Z}^o}  \frac{\partial \mathbf{Z}^{o}}{\partial \mathbf{b}^o}
= \sum_{i=1}^N 1 \cdot (\mathbf{\hat{y}}_i - \mathbf{y}_i) 
= \sum_{i=1}^N \delta_{oi}$$

In [ ]:
function output_bias_gradient(loss_grad)
    sum(loss_grad, dims=1)
end;

### Hidden layer gradients

$$\delta_{h} 
= \frac{\partial J}{\partial \mathbf{Z}^h} 
= \frac{\partial J}{\partial \mathbf{H}} \frac{\partial \mathbf{H}}{\partial \mathbf{Z}^h} 
= \frac{\partial J}{\partial \mathbf{Z}^o} \frac{\partial \mathbf{Z}^o}{\partial H} \frac{\partial H}{\partial \mathbf{Z}^h}$$

In [ ]:
function hidden_layer_gradient(H, out_weights, loss_grad)
    H .* (1 .- H) .* (loss_grad * out_weights')
end

#### Hidden Weight Gradient

$$
\frac{\partial J}{\partial \mathbf{W}^h} = \mathbf{X}^T \cdot \delta^{h}
$$

In [ ]:
function hidden_weight_gradient(X, hidden_layer_grad)
    X' * hidden_layer_grad
end;

#### Hidden Bias Gradient

$$
\frac{\partial \xi}{\partial \mathbf{b}_{h}} 
= \frac{\partial \xi}{\partial H} \frac{\partial H}{\partial Z_{h}} \frac{\partial Z_{h}}{\partial \mathbf{b}_{h}}
= \sum_{j=1}^N \delta_{hj}
$$

In [ ]:
function hidden_bias_gradient(hidden_layer_grad)
    sum(hidden_layer_grad, dims=1)
end;

## Initialize Weights

In [ ]:
function initialize_weights()
    hidden_weights = randn(2, 3)
    hidden_bias = randn(1, 3)
    output_weights = randn(3, 2)
    output_bias = randn(1, 2)
    # hidden_weights, hidden_bias, output_weights, output_bias
    [hidden_weights, hidden_bias, output_weights, output_bias]
end;

## Compute Gradients

In [ ]:
function compute_gradients(X, y_true, w_h, b_h, w_o, b_o)
    hidden_activations = hidden_layer(X, w_h, b_h)
    y_hat = output_layer(hidden_activations, w_o, b_o)

    loss_grad = loss_gradient(y_hat, y_true)
    output_weight_grad = output_weight_gradient(hidden_activations, loss_grad)
    output_bias_grad = output_bias_gradient(loss_grad)

    hidden_layer_grad = hidden_layer_gradient(hidden_activations, w_o, loss_grad)
    hidden_weight_grad = hidden_weight_gradient(X, hidden_layer_grad)
    hidden_bias_grad = hidden_bias_gradient(hidden_layer_grad)

    [hidden_weight_grad, hidden_bias_grad, output_weight_grad, output_bias_grad]
end;

function compute_gradients(X, y_true, wab)
    w_h, b_h, w_o, b_o = wab
    hidden_activations = hidden_layer(X, w_h, b_h)
    y_hat = output_layer(hidden_activations, w_o, b_o)

    loss_grad = loss_gradient(y_hat, y_true)
    out_weight_grad = output_weight_gradient(hidden_activations, loss_grad)
    out_bias_grad = output_bias_gradient(loss_grad)

    hidden_layer_grad = hidden_layer_gradient(hidden_activations, w_o, loss_grad)
    hidden_weight_grad = hidden_weight_gradient(X, hidden_layer_grad)
    hidden_bias_grad = hidden_bias_gradient(hidden_layer_grad)

    hidden_weight_grad, hidden_bias_grad, out_weight_grad, out_bias_grad
end;

## Check Gradients

It's easy to make mistakes with the numerous inputs to the backpropagation algorithm. A simple way to test for accuracy is to compare the change in the output for slightly perturbed parameter values with the change implied by the computed gradient (see [here](http://ufldl.stanford.edu/wiki/index.php/Gradient_checking_and_advanced_optimization) for more detail).

In [ ]:
# change individual parameters by +/- epsilon
ϵ = 1e-4;

# initialize weights and biases
params = initialize_weights();

# Get all parameter gradients
grad_params = compute_gradients(X, Y, params);

# Check each parameter matrix
for (i, param) ∈ enumerate(params)
    print("round $i\n")
    n_rows, n_cols = size(param)
    for row ∈ 1:n_rows
        for col ∈ 1:n_cols
            params_low = deepcopy(params)
            params_low[i][row, col] -= ϵ

            params_high = deepcopy(params)
            params_high[i][row, col] += ϵ

            # Compute the numerical gradient
            loss_high = loss(forward_prop(X, params_high), Y)
            loss_low = loss(forward_prop(X, params_low), Y)

            numerical_gradient = (loss_high - loss_low) / 2ϵ
            backprop_gradient = grad_params[i][row, col]

            print("numerical_gradient: $numerical_gradient \nbackprop_gradient $backprop_gradient \n\n")

            @assert isapprox(numerical_gradient, backprop_gradient)
        end
    end
end

## Train Network

In [ ]:
function update_momentum(
    X,
    y_true,
    param_list,
    Ms,
    momentum_term,
    learning_rate)
    """Update the momentum matrices."""

    gradients = compute_gradients(X, y_true, param_list)
    [momentum_term .* momentum .- learning_rate .* grads for (momentum, grads) in zip(Ms, gradients)]
end;

In [ ]:
function update_params(param_list, Ms)
    """Update the parameters."""
    [P .+ M for (P, M) in zip(param_list, Ms)]
end;

In [ ]:
function train_network(; iterations=1000, lr=0.01, mf=0.1)

    # Initialize weights and biases
    param_list = initialize_weights()

    # Momentum Matrices = [MWh, Mbh, MWo, Mbo]
    Ms = [zeros(Float64, size(M)) for M ∈ param_list]

    train_loss = [loss(forward_prop(X, param_list), Y)]
    for i ∈ 1:iterations
        if mod(i, 1000) == 0
            print("iteration $i ... \n")
        end
        # Update the moments and the parameters
        Ms = update_momentum(X, Y, param_list, Ms, mf, lr)

        param_list = update_params(param_list, Ms)

        append!(train_loss, loss(forward_prop(X, param_list), Y))
    end

    param_list, train_loss
end;

In [ ]:
trained_params, train_loss = train_network(
    iterations=n_iterations,
    lr=learning_rate,
    mf=momentum_factor);

In [ ]:
hidden_weights, hidden_bias, output_weights, output_bias = trained_params

### Plot Training Loss

This plot displays the training loss over 50K iterations for 50K training samples with a momentum term of 0.5 and a learning rate of 1e-4. 

It shows that it takes over 5K iterations for the loss to start to decline but then does so very fast. We have not uses stochastic gradient descent, which would have likely significantly accelerated convergence.

In [ ]:
train_loss_per_iteration_df = DataFrame(
    Iteration=collect(1:n_iterations+1),
    LogLoss=log.(train_loss),
);

In [ ]:

# ax = PlotlyJS.plot(
#     train_loss_per_iteration_df,
#     x=:Iteration,
#     y=:LogLoss,
#     m=1,
#     kind="scatter",
#     mode="lines",
#     labels=Dict(
#         :Iteration => "Iteration",
#         :LogLoss => "Log(ξ)",
#     ),
#     PlotlyJS.Layout(
#         title="Log(ξ) / Iteration",
#         width=800, height=500,
#     )
# )


## Decision Boundary

The following plots show the function learned by the neural network with a three-dimensional hidden layer form two-dimensional data with two classes that are not linearly separable as shown on the left. The decision boundary misclassifies very few data points and would further improve with continued training. 

The second plot shows the representation of the input data learned by the hidden layer. The network learns hidden layer weights so that the projection of the input from two to three dimensions enables the linear separation of the two classes. 

The last plot shows how the output layer implements the linear separation in the form of a cutoff value of 0.5 in the output dimension.

In [ ]:
function meshgrid(x, y)
    mg = Iterators.product(x, y)

    xx = first.(mg)
    yy = last.(mg)

    xx, yy
end;

In [ ]:
# 500 instead of 200 for more resulotion boundry in the coutour plot.
n_vals = 500;
x1 = range(-1.5, 1.5, n_vals);
x2 = range(-1.5, 1.5, n_vals);
# create the grid
xx, yy = meshgrid(x1, x2);

In [ ]:
# Initialize and fill the feature space
feature_space = zeros(n_vals, n_vals);
for i ∈ 1:n_vals
    for j ∈ 1:n_vals
        X_ = [xx[i, j] yy[i, j]]
        probabilities = predict(X_, trained_params)
        index = argmax(probabilities)
        feature_space[i, j] = index[2] - 1
    end
end

In [ ]:
df = transform(df, :Label => ByRow(label -> label == 0 ? "Outer Circle" : "Inner Circle") => :State);

alpha = 0.15;
colorscale = [[0, "rgba(0,0,255,$alpha)"], [1, "rgba(255,255,0,$alpha)"]]

PlotlyJS.plot([
        contour(
            x=x1, # horizontal axis
            y=x2, # vertical axis
            z=feature_space', # data

            # contours_coloring="lines",

            # autocolorscale=true,
            colorscale=colorscale,

            line_width=1,
            line_smoothing=0.85,
            colorbar=attr(
                title="Decision Boundry", # title here
                titleside="right",
                titlefont=attr(
                    size=14,
                    family="Arial, sans-serif"),
                thickness=25,
                thicknessmode="pixels",
                len=0.8,
                lenmode="fraction",
                outlinewidth=0,
            ),

            # Smooth Coloring based on Z (In this case Z = 0, 1)
            contours=attr(
                coloring ="heatmap",
                # showlabels = true, # show labels on contours
                labelfont = attr( # label font properties
                    size = 12,
                    color = "white",
                )
            )
        ),
        
        PlotlyJS.scatter(
            df,
            x=:X1,
            y=:X2,

            # NOTE: marker_color for scatter() and color for plot()
            marker_color=:Label,
            
            text=:State,
            
            mode="markers",
            labels=Dict(
                :X1 => "X",
                :X2 => "Y",
                :Label => "Labels"
            ),
            marker=attr(size=8, line=attr(width=1, color="DarkSlateGrey")),
        ),
        ],
    PlotlyJS.Layout(
        title="Circles Dataset Visualization",
        width=700, height=700,
    )
)

## Projection on Hidden Layer

In [ ]:
# 500 instead of 200 for more resulotion boundry in the coutour plot.
n_vals = 25;
x1 = range(-1.5, 1.5, n_vals);
x2 = range(-1.5, 1.5, n_vals);
# create the grid
xx, yy = meshgrid(x1, x2);

X_ = [vec(xx) vec(yy)]'

In [ ]:
h_ = hidden_layer(X, hidden_weights, hidden_bias);

hidden_layer_output_df = DataFrame(
    X=h_[:, 1],
    Y=h_[:, 2],
    Z=h_[:, 3],
    Label=y[:, 1],
);

hidden_layer_output_df = transform(hidden_layer_output_df,
    :Label => ByRow(label -> label == 0 ? "Outer Circle" : "Inner Circle") => :State
);

plot(
    PlotlyJS.scatter(
        hidden_layer_output_df,
        x=:X,
        y=:Y,
        z=:Z,

        # NOTE: marker_color for scatter() and color for plot()
        marker_color=:Label, type="scatter3d", text=:State,
        mode="markers",
        labels=Dict(
            :Label => "Labels"
        ),
        marker=attr(size=8, line=attr(width=1, color="DarkSlateGrey")),
    ),
    Layout(
        title="Hidden Layers Outputs",
        width=700, height=700,
    ),
)

## Network Output Surface Plot

In [ ]:
zz = forward_prop(X_', hidden_weights, hidden_bias, output_weights, output_bias);
zz = reshape(zz[:, 2], 25, 25);

In [ ]:
plot(
    PlotlyJS.surface(
        z=zz,
        x=xx,
        y=yy,
        contours_z=attr(
            show=true,
            usecolormap=true,
            highlightcolor="limegreen",
            project_z=true
        ),),
    Layout(
        title="Mt Bruno Elevation", autosize=false,
        width=700, height=700,
    ),
)

## Summary

To sum up: we have seen how a very simple network with a single hidden layer with three nodes and a total of 17 parameters is able to learn how to solve a non-linear classification problem using backprop and gradient descent with momentum. 

We will next review key design choices useful to design and train more complex architectures before we turn to popular deep learning libraries that facilitate the process by providing many of these building blocks and automating the differentiation process to compute the gradients and implement backpropagation.